In [1]:
'''
Created on Mar 18, 2016
@author: pushkar
'''

import urllib2
from bs4 import BeautifulSoup
import csv
import time
import re


def getAmazonDetails(isbn):
    
    with open('csv_files/amazon_book_ratings.csv', 'a') as csvfile_ratings, open('csv_files/amazon_book_reviews.csv', 'a') as csvfile_reviews:
        ##Create file headers and writer
        ratings_fieldnames = ['book_isbn', 'avg_rating', 'five_rating', 'four_rating', 'three_rating', 'two_rating', 'one_rating' ]
        writer = csv.DictWriter(csvfile_ratings, delimiter=',', lineterminator='\n', fieldnames=ratings_fieldnames)
        ##writer.writeheader()
         
        reviews_fieldnames = ['book_isbn', 'review']            
        writer_book = csv.DictWriter(csvfile_reviews, delimiter=',', lineterminator='\n', fieldnames=reviews_fieldnames)
        ##writer_book.writeheader()

        ##Get Overall details of the book    
        req = urllib2.Request('http://www.amazon.com/product-reviews/' + isbn + '?ie=UTF8&showViewpoints=1&sortBy=helpful&pageNumber=1', headers={ 'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11' })
        html = urllib2.urlopen(req).read()
        soup = BeautifulSoup(html, 'html.parser')
    
        avgRatingTemp = soup.find_all('div',{'class':"a-row averageStarRatingNumerical"})[0].get_text()
        avgRating = re.findall('\d+\.\d+', avgRatingTemp)[0]
    
        try:
            fiveStarRatingTemp = soup.find_all('a',{'class':"a-size-small a-link-normal 5star histogram-review-count"})[0].get_text()
            fiveStarRating = fiveStarRatingTemp.strip('%')
        except:
            fiveStarRating = 0

        try:
            fourStarRatingTemp = soup.find_all('a',{'class':"a-size-small a-link-normal 4star histogram-review-count"})[0].get_text()
            fourStarRating = fourStarRatingTemp.strip('%')
        except:
            fourStarRating = 0

        try:
            threeStarRatingTemp = soup.find_all('a',{'class':"a-size-small a-link-normal 3star histogram-review-count"})[0].get_text()
            threeStarRating = threeStarRatingTemp.strip('%')
        except:
            threeStarRating = 0

        try:
            twoStarRatingTemp = soup.find_all('a',{'class':"a-size-small a-link-normal 2star histogram-review-count"})[0].get_text()
            twoStarRating = twoStarRatingTemp.strip('%')
        except:
            twoStarRating = 0

        try:
            oneStarRatingTemp = soup.find_all('a',{'class':"a-size-small a-link-normal 1star histogram-review-count"})[0].get_text()
            oneStarRating = oneStarRatingTemp.strip('%')
        except:
            oneStarRating = 0

        writer.writerow({'book_isbn': isbn, 'avg_rating': avgRating, 'five_rating': fiveStarRating, 
                         'four_rating': fourStarRating, 'three_rating': threeStarRating, 'two_rating': twoStarRating,
                         'one_rating': oneStarRating})
    
        ##Get top 20 helpful review of book
        for pagenumber in range(1,3):
            req = urllib2.Request('http://www.amazon.com/product-reviews/' + isbn + '?ie=UTF8&showViewpoints=1&sortBy=helpful&pageNumber='+ str(pagenumber), headers={ 'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11' })
            html = urllib2.urlopen(req).read()
            soup = BeautifulSoup(html, 'html.parser')    
            for i in range(0,10):
                try:
                    review = soup.find_all('div',{'class':"a-section review"})[i].contents[3].get_text().encode('UTF-8')
                    #print review
                    writer_book.writerow({'book_isbn': isbn, 'review': review})
                except:
                    print "No Reviews ISBN - " + isbn
                
#getAmazonDetails('0940650703')

In [40]:
import urllib2
import csv
import time
import sys
import xml.etree.ElementTree as ET
import os
import traceback
import random
from IPython.display import clear_output

def getval(root, element):
    try:
        ret = root.find(element).text
        if ret is None:
            return ""
        else:
            return ret.encode("utf8")
    except:
        return ""
    

with open('csv_files/amazon_book_ratings.csv', 'w') as csvfile_ratings, open('csv_files/amazon_book_reviews.csv', 'w') as csvfile_reviews:
    ##Create file headers and writer
    ratings_fieldnames = ['book_isbn', 'avg_rating', 'five_rating', 'four_rating', 'three_rating', 'two_rating', 'one_rating' ]
    writer = csv.DictWriter(csvfile_ratings, delimiter=',', lineterminator='\n', fieldnames=ratings_fieldnames)
    writer.writeheader()
         
    reviews_fieldnames = ['book_isbn', 'review']            
    writer_book = csv.DictWriter(csvfile_reviews, delimiter=',', lineterminator='\n', fieldnames=reviews_fieldnames)
    writer_book.writeheader()

with open('csv_files/user_data.csv', 'w') as csvfile, open('csv_files/book_data.csv', 'w') as csvfile_book, open('csv_files/book_author.csv', 'w') as csvfile_author:
    fieldnames = ['id', 'name','user_name', 'profile_url','image_url', 'about', 'age', 'gender', 
                  'location','joined','last_active' ]
    writer = csv.DictWriter(csvfile, delimiter = ',', lineterminator = '\n', fieldnames=fieldnames)
    writer.writeheader()
    book_fieldnames = [
                        'user_id',
                        'b_id',
                        'shelf',
                        'isbn', 
                        'isbn13',
                        'text_reviews_count',
                        'title',
                        'image_url',
                        'link',
                        'num_pages',
                        'b_format',
                        'publisher',
                        'publication_day', 
                        'publication_year', 
                        'publication_month',
                        'average_rating', 
                        'ratings_count', 
                        'description', 
                        'published',
                        'children',
                        'religion',
                        'history',
                        'math',
                        'anatology',
                        'poetry',
                        'encyclopedia',
                        'dictionaries',
                        'comics',
                        'art',
                        'cookbook',
                        'diaries',
                        'journals',
                        'prayer_books',
                        'series',
                        'trilogy',
                        'biographies',
                        'autobiographies',
                        'fantasy',
                        'comic',
                        'self-help',
                        'science-fiction',
                        'non-fiction']
  
    
    writer_book = csv.DictWriter(csvfile_book, delimiter = ',', lineterminator = '\n', fieldnames=book_fieldnames)
    writer_book.writeheader()
    author_fieldnames = [
                        'u_id',
                        'b_id',
                        'a_id',
                        'name',
                        'average_rating',
                        'ratings_count',
                        'text_reviews_count']
    writer_author = csv.DictWriter(csvfile_author, delimiter = ',', lineterminator = '\n', fieldnames = author_fieldnames)
    writer_author.writeheader()

    lst = []
    i = 0
    
    while i < 1000:   
        try:     
            #clear_output()
            #time.sleep(1)
            clear_output()
            c = random.randint(5000000, 5625000) #7500000 625000
            print "random number: " + str(c)    

            if (c not in lst):
                print "getting information for user id:"+ str(c)
                lst.append(c)
                url = 'https://www.goodreads.com/user/show/'+ str(c) +'.xml?key=i3Zsl7r13oHEQCjv1vXw'
                response = urllib2.urlopen(url)
                user_data_xml = response.read()
                #write xml to file
                print "User number:" + str(i)
                i = i + 1
                f = open("xml_docs/user"+ str(c) +".xml", "w")
                try:
                    f.write(user_data_xml)
                finally:
                    f.close()
                #root = ET.fromstring()
                root = ET.parse("xml_docs/user"+ str(c) +".xml").getroot()
                os.remove("xml_docs/user"+ str(c) +".xml")
                user_element = root.find('user')
                id = getval(user_element,'id')
                name = getval(user_element,'name')
                user_name = getval(user_element,'user_name')
                profile_url = getval(user_element,'link')
                image_url = getval(user_element,'image_url')
                about = getval(user_element,'about')
                age = getval(user_element,'age')
                gender = getval(user_element,'gender')
                location = getval(user_element,'location')
                joined = getval(user_element,'joined')
                last_active = getval(user_element,'last_active')
                writer.writerow({'id': id, 'name' : name,'user_name' : user_name,
                                  'profile_url' : profile_url,'image_url' : image_url,
                                 'about' : about, 'age': age, 'gender' : gender, 
                                 'location' : location, 'joined' : joined, 'last_active': last_active})
                print "Saved user data for user id:" + str(c)
            
            
                # get list of user shelves
            
                user_shelves_root =  user_element.find('user_shelves')
            
                user_shelf_list = []
            
                for user_shelf in user_shelves_root.findall("user_shelf"):
                    shelf = getval(user_shelf,"name")
                    
                    #Books on Shelf
                    print "Checking for books in shelf: " + shelf + " for user id:" + str(c)
                    
                    shelf_url = "https://www.goodreads.com/review/list/"+ str(c) +".xml?key=i3Zsl7r13oHEQCjv1vXw&v=2&shelf=" + shelf
                    #time.sleep(1)
                    print shelf_url
                    response = urllib2.urlopen(shelf_url)
                    shelf_data_xml = response.read()
                    # write xml to file
                    f = open("xml_docs/user_shelf_" + shelf + "_"+ str(c) + ".xml", "w")
                    try:
                        f.write(shelf_data_xml)
                    finally:
                        f.close()
                    
                    shelf_root = ET.parse("xml_docs/user_shelf_" + shelf + "_"+ str(c) + ".xml").getroot()
                    
                    os.remove("xml_docs/user_shelf_" + shelf + "_"+ str(c) + ".xml")
                    reviews = shelf_root.find("reviews")
                    for review in reviews.findall("review"):
                        for book in review.findall("book"):
                            b_id = getval(book,"id")
                            isbn = getval(book,"isbn")
                            print "Fetching data for book with isbn:" + str(isbn) + " and id:" + str(id)
                            isbn13 = getval(book,"isbn13")
                            text_reviews_count = getval(book,"text_reviews_count")
                            title = getval(book,"title")
                            image_url = getval(book,"image_url")
                            link = getval(book,"link")
                            num_pages = getval(book,"num_pages")
                            b_format = getval(book,"format")
                            publisher = getval(book,"publisher")
                            publication_day = getval(book,"publication_day")
                            publication_year = getval(book, "publication_year") 
                            publication_month = getval(book,"publication_month")
                            average_rating = getval(book,"average_rating")
                            ratings_count = getval(book,"rating_count")
                            description = getval(book,"description")
                            published = getval(book,"published")
                            
                            #get number of books on each type of shelf
                            book_url = 'https://www.goodreads.com/book/show/'+str(b_id)+'.xml?key=i3Zsl7r13oHEQCjv1vXw'
                            response = urllib2.urlopen(book_url)
                            book_data_xml = response.read()
                            # write xml to file
                            f = open("xml_docs/book_data_" + str(b_id) + ".xml", "w")
                            try:
                                f.write(book_data_xml)
                            finally:
                                f.close()
                    
                            book_root = ET.parse("xml_docs/book_data_" + str(b_id) + ".xml").getroot()
                            os.remove("xml_docs/book_data_" + str(b_id) + ".xml")
                            print "checking count in shelf for book_id:" + str(b_id) 
                            book_root = book_root.find("book")
                            book_shelves = book_root.find("popular_shelves")
                            children = 0
                            religion = 0
                            history = 0
                            math = 0
                            anatology = 0
                            poetry = 0
                            encyclopedia = 0
                            dictionaries = 0
                            comics = 0
                            art = 0
                            cookbook = 0
                            diaries = 0
                            journals = 0
                            prayer_books = 0
                            series = 0
                            trilogy = 0
                            biographies = 0
                            autobiographies = 0
                            fantasy = 0 
                            comic = 0
                            self_help = 0
                            science_fiction = 0 
                            non_fiction = 0
                            
                            for shelf_type in book_shelves.findall("shelf"):
                                attributes = shelf_type.attrib
                                name = attributes['name']
                                count = attributes['count']
                                print name + ":" + count
                                if(name == 'children'):
                                    children = count
                                
                                if(name =='religion'):
                                        religion = count
                                if(name =='science-fiction'):
                                        science_fiction = count
                                if(name =='history'):
                                        history = count
                                if(name =='math'):
                                        math = count
                                if(name =='anatology'):
                                        anatology = count
                                if(name =='poetry'):
                                        poetry = count
                                if(name =='encyclopedia'):
                                        encyclopedia = count
                                if(name =='dictionaries'):
                                        dictionaries = count
                                if(name =='comics'):
                                        comics = count
                                if(name =='art'):
                                    art = count
                                if(name =='cookbook'):
                                    cookbook = count
                                if(name =='diaries'):
                                    diaries = count
                                if(name =='journals'):
                                    journals = count
                                if(name =='prayer-books'):
                                    prayer_books = count
                                if(name =='series'):
                                    series = count
                                if(name =='trilogy'):
                                    trilogy = count
                                if(name =='biographies'):
                                    biortaphies = count
                                if(name =='autobiographies'):
                                    autobiographies = count
                                if(name =='fantasy'):
                                    fantasy = count
                                if(name =='comic'):
                                    comic = count
                                if(name =='self-help'):
                                    self_help = count
                                if(name == 'non-fiction'):
                                    non_fiction = count
                                
                                
                            writer_book.writerow({
                                'user_id': id,
                                'b_id' : b_id ,
                                'shelf' : shelf,
                                'isbn' : isbn, 
                                'isbn13': isbn13,
                                'text_reviews_count' : text_reviews_count,
                                'title' : title,
                                'image_url' : image_url,
                                'link' : link,
                                'num_pages' : num_pages,
                                'b_format' : b_format,
                                'publisher' : publisher,
                                'publication_day' : publication_day, 
                                'publication_year' : publication_year, 
                                'publication_month' : publication_month,
                                'average_rating' : average_rating, 
                                'ratings_count' : ratings_count, 
                                'description' : description, 
                                'children' : children,
                                'religion' : religion,
                                'history' : history,
                                'math': math,
                                'anatology' : anatology,
                                'poetry' : poetry,
                                'encyclopedia' : encyclopedia,
                                'dictionaries' : dictionaries,
                                'comics' : comics,
                                'art' : art,
                                'cookbook' : cookbook,
                                'diaries' : diaries,
                                'journals' : journals,
                                'prayer_books' : prayer_books,
                                'series' : series ,
                                'trilogy' : trilogy,
                                'biographies' : biographies ,
                                'autobiographies' : autobiographies,
                                'fantasy' : fantasy,
                                'comic' : comic,
                                'self-help' : self_help,
                                'science-fiction' : science_fiction,
                                'non-fiction' : non_fiction})

                            print "Data written on csv for book:" + title
                            getAmazonDetails(isbn)
                            print "Fetched review data from Amazon for book :" + title
                           

                            authors = book.find("authors")
                            for author in authors.findall("author"):
                                a_id = getval(author,"id")
                                name = getval(author,"name")
                                average_rating = getval(author,"average_rating")
                                ratings_count = getval(author,"ratings_count")
                                text_reviews_count = getval(author,"text_reviews_count")
                                writer_author.writerow({'u_id': id,
                                                        'b_id' : b_id,
                                                        'a_id' : a_id,
                                                        'name' : name,
                                                        'average_rating' : average_rating,
                                                        'ratings_count' : ratings_count,
                                                        'text_reviews_count' : text_reviews_count})
        except:
            #time.sleep(1)
            print "Exception!!"
            print traceback.format_exc()
    print "End of Program"

random number: 5536323
getting information for user id:5536323
User number:999
Saved user data for user id:5536323
Checking for books in shelf: read for user id:5536323
https://www.goodreads.com/review/list/5536323.xml?key=i3Zsl7r13oHEQCjv1vXw&v=2&shelf=read
Fetching data for book with isbn:0440241413 and id:5536323
checking count in shelf for book_id:9416
to-read:12116
chick-lit:4471
fiction:1780
currently-reading:1574
romance:1097
favorites:1063
books-i-own:620
chicklit:616
series:537
contemporary:517
humor:463
sophie-kinsella:387
owned:380
chic-lit:337
adult:292
funny:223
adult-fiction:204
owned-books:177
contemporary-romance:161
comedy:143
humour:141
favourites:117
fluff:116
british:114
guilty-pleasures:112
library:108
kindle:93
my-books:93
i-own:92
young-adult:90
novels:90
shopaholic:87
beach-reads:86
contemporary-fiction:83
general-fiction:83
realistic-fiction:81
to-buy:76
default:72
ebook:72
girly:64
borrowed:62
movie:61
movies:61
book-club:59
read-in-2009:58
english:57
ebooks:5

In [25]:
f = open("xml_docs/book_data_" + "520980" + ".xml", "w")
try:
    f.write(book_data_xml)
finally:
    f.close()
    book_root = ET.parse("xml_docs/book_data_" + "520980" + ".xml").getroot()
    #os.remove("xml_docs/book_data_" + str(b_id) + ".xml")
    print "checking count in shelf for book_id:" + "520980" 
    book_shelves = book_root.find("book")
    book_in_shelf = book_shelves.find("popular_shelves")
   
    children = 0
    religion = 0
    history = 0
    math = 0
    anatology = 0
    poetry = 0
    encyclopedia = 0
    dictionaries = 0
    comics = 0
    art = 0
    cookbook = 0
    diaries = 0
    journals = 0
    prayer_books = 0
    series = 0
    trilogy = 0
    biographie = 0
    autobiographies = 0
    fantasy = 0 
                            
    print book_in_shelf.findall("shelf")                        
    for shelf in book_in_shelf.findall("shelf"):
        print shelf
                               

checking count in shelf for book_id:520980
[<Element 'shelf' at 0x544f4e0>]
<Element 'shelf' at 0x544f4e0>


In [ ]:
 attributes = shelf.attrib
                                name = attributes['name']
                                print name
                                count = attributes['count']
                                print count
                                if(name == 'children'):
                                    children = count
                                
                                if(name =='religion'):
                                        religion = count
                                
                                if(name =='history'):
                                        history = count
                                if(name =='math'):
                                        math = count
                                if(name =='anatology'):
                                        anatology = count
                                if(name =='poetry'):
                                        poetry = count
                                if(name =='encyclopedia'):
                                        encyclopedia = count
                                if(name =='dictionaries'):
                                        dictionaries = count
                                if(name =='comics'):
                                        comics = count
                                if(name =='art'):
                                    art = count
                                if(name =='cookbook'):
                                    cookbook = count
                                if(name =='diaries'):
                                    diaries = count
                                if(name =='journals'):
                                    journals = count
                                if(name =='prayer-books'):
                                    prayer_books = count
                                if(name =='series'):
                                    series = count
                                if(name =='trilogy'):
                                    trilogy = count
                                if(name =='biographies'):
                                    biortaphies = count
                                if(name =='autobiographies'):
                                    autobiographies = count
                                if(name =='fantasy'):
                                    fantasy = count
                                
                            